# Bag of Words Text Classifier

In [ ]:
import torch

import random
import torch.nn as nn

In [ ]:



!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/dev.txt
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/test.txt
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/train.txt


!mkdir data data/classes
!cp dev.txt data/classes
!cp test.txt data/classes
!cp train.txt data/classes

In [ ]:

def _read_data(filename):

    Data = []
    with open( filename,'r') as f:
        for Line in f:
                Line = Line.lower().strip()
                Line = Line.split(' ||| ' )

                Data.append(Line)
    return Data

train_data=_read_data('data/classes/train.txt')
test_data=_read_data('data/classes/test.txt')

### Construct the Vocab and Datasets

In [ ]:
# creating the word and tag indices
WORDTOINDEX = {}
WORDTOINDEX["<unk>"] = len(WORDTOINDEX)

TAGTOINDEX = {}
# create word to index dictionary and tag to index dictionary from data
def _create_dict(data, check_unk=False):
    for _line in data:
        for word in _line[1].split(" "):
            if check_unk == False:
                if word not in WORDTOINDEX:
                      WORDTOINDEX[word] = len(WORDTOINDEX)
            else:
                if word not in WORDTOINDEX:
                      WORDTOINDEX[word] = WORDTOINDEX["<unk>"]

        if _line[0] not in TAGTOINDEX:
            TAGTOINDEX[_line[0]] = len(TAGTOINDEX)

_create_dict(train_data)
_create_dict(test_data, check_unk = True)

# create word and tag tensors from data
def _create_tensor(data):
    for _line in data:
        yield([WORDTOINDEX[word] for word in _line[1].split(" ")], TAGTOINDEX[_line[0]])

train_data = list(_create_tensor(train_data))
test_data = list(_create_tensor(test_data))

number_of_words = len(WORDTOINDEX)
number_of_tags = len(TAGTOINDEX)

### Model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# create a simple neural network with embedding layer, bias, and xavier initialization
class BoW(torch.nn.Module):
  def __init__(self, nwords, ntags):
      super(BoW, self).__init__()
      self.Embedding = nn.Embedding(nwords, ntags)
      nn.init.xavier_uniform_(self.Embedding.weight)

      type = torch.cuda.FloatTensor if torch.cuda.is_available( ) else torch.FloatTensor
      self.bias = torch.zeros(ntags, requires_grad = True).type(type)

  def forward(self, x):
      emb = self.Embedding(x)
      out = torch.sum(emb, dim = 0)+self.bias
      out = out.view(1, -1)
      return out

### Pretest the Model

In [ ]:
# function to convert sentence into tensor using word_to_index dictionary
def sentence_to_tensor(sentence):
    return torch.LongTensor([WORDTOINDEX[_word] for _word in sentence.split(" ")])
type = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
out = sentence_to_tensor("i love dogs").type(type)
test_model = BoW(number_of_words, number_of_tags).to(device)

test_model(out)

### Train the Model

In [ ]:
# train and test the BoW model
model = BoW(number_of_words, number_of_tags).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters( ))
type = torch.LongTensor

if torch.cuda.is_available():
    model.to(device)
    type = torch.cuda.LongTensor
# perform training of the Bow model
def train_bow(model,
              optimizer,
              criterion,
              train_data):
    for ITER in range(10):
        # perform training
        model.train()
        random.shuffle(train_data)
        total_loss = 0.0
        train_correct = 0
        for sentence, tag in train_data:
            sentence = torch.tensor(sentence).type(type)
            tag = torch.tensor([tag]).type(type)
            output = model(sentence)
            predicted = torch.argmax(output.data.detach()).item()

            loss = criterion(output, tag)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if predicted == tag: train_correct+=1
        # perform testing of the model
        model.eval()
        test_correct = 0
        for _sentence, tag in test_data:
            _sentence = torch.tensor(_sentence).type(type)
            output = model(_sentence)
            predicted = torch.argmax(output.data.detach()).item()
            if predicted == tag: test_correct += 1


        # print model performance results
        log = f'ITER: {ITER+1} | ' \
            f'train loss/sent: {total_loss/len(train_data):.4f} | ' \
            f'train accuracy: {train_correct/len(train_data):.4f} | ' \
            f'test accuracy: {test_correct/len(test_data):.4f}'
        print(log)
# call the train_bow function
train_bow(model, optimizer, criterion, train_data)

### Perform inference

In [ ]:

def perform_inference(model, sentence, word_to_index, tag_to_index, device):
    """
    Perform inference on the trained BoW model.

    Args:
        model (torch.nn.Module): The trained BoW model.
        sentence (str): The input sentence for inference.
        word_to_index (dict): A dictionary mapping words to their indices.
        tag_to_index (dict): A dictionary mapping tags to their indices.
        device (str): "cuda" or "cpu" based on availability.

    Returns:
        str: The predicted class/tag for the input sentence.
    """
    # Preprocess the input sentence to match the model's input format
    sentence_tensor = sentence_to_tensor(sentence, word_to_index)

    # Move the input tensor to the same device as the model
    sentence_tensor = sentence_tensor.to(device)
    
    # Make sure the model is in evaluation mode and on the correct device
    model.eval()
    model.to(device)

    # Perform inference
    with torch.no_grad():
        output = model(sentence_tensor)

    # Move the output tensor to CPU if it's on CUDA
    if device == "cuda":
        output = output.cpu()

    # Convert the model's output to a predicted class/tag
    predicted_class = torch.argmax(output).item()

    # Reverse lookup to get the tag corresponding to the predicted class
    for tag, index in tag_to_index.items():
        if index == predicted_class:
            return tag

    # Return an error message if the tag is not found
    return "Tag not found"

# Example usage:
# Load your trained model (model) and dictionaries (word_to_index and tag_to_index) here
# model = load_trained_model()
# word_to_index, tag_to_index = load_dictionaries()

# Determine the device based on availability
device = "cuda" if torch.cuda.is_available() else "cpu"

sample_sentence = "I love programming"
predicted_tag = perform_inference(model, sample_sentence, word_to_index, tag_to_index, device)
print(f"Predicted Tag: {predicted_tag}")